In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc

import keras as k
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Activation, BatchNormalization
from keras.callbacks import EarlyStopping
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Input
from keras import backend as K
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import cv2
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import fbeta_score, precision_score 
from skimage import io,transform

import time

import os
import fnmatch

Using TensorFlow backend.
ERROR:tornado.application:Exception in callback functools.partial(<function wrap.<locals>.null_wrapper at 0x7fd6c52621e0>)
Traceback (most recent call last):
  File "/home/cs231n/myVE35/lib/python3.5/site-packages/tornado/ioloop.py", line 600, in _run_callback
    ret = callback()
  File "/home/cs231n/myVE35/lib/python3.5/site-packages/tornado/stack_context.py", line 275, in null_wrapper
    return fn(*args, **kwargs)
  File "/home/cs231n/myVE35/lib/python3.5/site-packages/ipykernel/kernelbase.py", line 242, in enter_eventloop
    self.eventloop(self)
  File "/home/cs231n/myVE35/lib/python3.5/site-packages/ipykernel/eventloops.py", line 163, in loop_tk
    kernel.timer = Timer(doi)
  File "/home/cs231n/myVE35/lib/python3.5/site-packages/ipykernel/eventloops.py", line 151, in __init__
    self.app = Tk()
  File "/usr/lib/python3.5/tkinter/__init__.py", line 1871, in __init__
    self.tk = _tkinter.create(screenName, baseName, className, interactive, wantobjects

In [2]:
datagen = ImageDataGenerator(
    rotation_range=90,
    fill_mode='reflect',
    horizontal_flip=True,
    vertical_flip=True)

def multilabelmetrics(y_true,y_pred):
    '''y_true and y_pred should be boolean np arrays
    of shape num_example x num_classes '''
    total = np.sum(y_true,axis = 0)
    tp = np.sum(y_true*y_pred,axis=0)
    tn = np.sum((1-y_true)*(1-y_pred),axis=0)
    fp = np.sum((1-y_true)*y_pred,axis=0)
    fn = np.sum(y_true*(1-y_pred),axis=0)
    return total,tp,tn,fp,fn

def combine_predictions(x,y,y1,y2,thresh,thresh1,thresh2,thresh3):
    y_pred = np.zeros((x.shape[0],17),np.uint8)
    y_bool = np.array((y > thresh),np.uint8)
    y1_bool = np.array((y1 > thresh1),np.uint8)
    y2_bool = np.array((y2 > thresh2)*np.tile(y1[:,0]>thresh3,(7,1)).T,np.uint8)
    y_pred[:,:7] = y2_bool
    y_pred[:,7:13] = y1_bool[:,1:]
    y_pred[:,13:] = y_bool
    return y_pred

def combine_predictions_2(x,y,y1,y2,thresh,thresh1,thresh2,thresh3,thresh4):#if cloud > thresh4, 
    y_pred = np.zeros((x.shape[0],17),np.uint8)
    y_bool = np.array((y > thresh),np.uint8)
    y1_bool = np.array((y1 > thresh1),np.uint8)
    y2_bool = np.array((y2 > thresh2)*np.tile(y1[:,0]>thresh3,(7,1)).T,np.uint8)
    y_pred[:,:7] = y2_bool
    y_pred[:,7:13] = y1_bool[:,1:]
    y_pred[:,13:] = y_bool
    y_pred[((np.nonzero(y[:,1]>thresh4))[0]),:] = 0
    y_pred[((np.nonzero(y[:,1]>thresh4))[0]),14] = 1
    return y_pred

callbacks = [EarlyStopping(monitor='val_loss', patience=2, verbose=0)]

In [3]:
x_train = np.zeros((40479,96,96,3), np.float32)
y_train = []

df_train = pd.read_csv('train_v2.csv')

labels = ['blow_down',
 'bare_ground',
 'conventional_mine',
 'blooming',
 'artisinal_mine',
 'selective_logging',         
 'slash_burn', 
 'cultivation',
 'habitation',
 'road',
 'agriculture',
 'water',
 'primary',
 'partly_cloudy', 
 'cloudy',
 'clear',
 'haze',]

label_map = {l: i for i, l in enumerate(labels)}
inv_label_map = {i: l for l, i in label_map.items()}

i=0

for f, tags in tqdm(df_train.values[:40479], miniters=1000):    
    img = cv2.imread('train-jpg/{}.jpg'.format(f))
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
    x_train[i,:,:,:] = np.array(cv2.resize(img, (96, 96)),np.float32)/255.#139 minimum size for inception
    i+=1
    y_train.append(targets)
  
y_train = np.array(y_train, np.uint8)

print(x_train.shape)
print(y_train.shape)

100%|██████████| 40479/40479 [01:01<00:00, 660.51it/s]

(40479, 96, 96, 3)
(40479, 17)


In [4]:
#subtracting mean
train_mean = np.mean(x_train,axis = 0)
x_train -= train_mean

In [5]:
#weather classifier (last four labels - mutually exclusive)
x_train, x_val, y_train_w, y_val_w = train_test_split(x_train,y_train[:,-4:],test_size=0.1)
print(x_train.shape)
print(y_train_w.shape)
print(x_val.shape)
print(y_val_w.shape)

(36431, 96, 96, 3)
(36431, 4)
(4048, 96, 96, 3)
(4048, 4)


In [6]:
model = Sequential()#using same architecture for all three models
model.add(Conv2D(32, (3, 3), padding = 'same', input_shape=(96, 96, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(48, (3, 3), padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(48, (3, 3), padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(48, (3, 3), padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3), padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3), padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(2048))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(1024))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(4, activation='softmax'))

In [7]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam',
              metrics=['accuracy'])    
model.fit_generator(datagen.flow(x_train,y_train_w, batch_size = 128), validation_data=(x_val, y_val_w),
                  epochs=10, steps_per_epoch=x_train.shape[0]/ 128, callbacks=callbacks,
                  )

Epoch 1/10
285/284 [==============================] - 360s - loss: 0.4192 - acc: 0.8575 - val_loss: 0.6770 - val_acc: 0.7927
Epoch 2/10
285/284 [==============================] - 354s - loss: 0.3071 - acc: 0.8898 - val_loss: 0.4610 - val_acc: 0.8478
Epoch 3/10
285/284 [==============================] - 354s - loss: 0.2824 - acc: 0.8976 - val_loss: 0.3937 - val_acc: 0.8434
Epoch 4/10
285/284 [==============================] - 354s - loss: 0.2610 - acc: 0.9029 - val_loss: 0.3494 - val_acc: 0.8673
Epoch 5/10
285/284 [==============================] - 354s - loss: 0.2527 - acc: 0.9073 - val_loss: 0.3955 - val_acc: 0.8691
Epoch 6/10
285/284 [==============================] - 354s - loss: 0.2479 - acc: 0.9100 - val_loss: 0.2244 - val_acc: 0.9222
Epoch 7/10
285/284 [==============================] - 355s - loss: 0.2336 - acc: 0.9132 - val_loss: 0.4080 - val_acc: 0.8708
Epoch 8/10
285/284 [==============================] - 354s - loss: 0.2324 - acc: 0.9152 - val_loss: 0.3334 - val_acc: 0.8846


In [8]:
y_pred = model.predict(x_val,batch_size=128)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val_w, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.929486048372
thresh: 0.1 	F2 score: 0.928134704969
thresh: 0.15 	F2 score: 0.924314476285
thresh: 0.2 	F2 score: 0.917907726332
thresh: 0.25 	F2 score: 0.91323757764
thresh: 0.3 	F2 score: 0.906373517787
thresh: 0.35 	F2 score: 0.901885704875


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [9]:
#continue with reduced learning rate
model.compile(loss='categorical_crossentropy', 
              optimizer=Adam(lr=0.0005),
              metrics=['accuracy']) 
model.fit_generator(datagen.flow(x_train,y_train_w, batch_size = 128), validation_data=(x_val, y_val_w),
                  epochs=10, steps_per_epoch=x_train.shape[0]/ 128, callbacks=callbacks,
                  )

Epoch 1/10
285/284 [==============================] - 355s - loss: 0.2187 - acc: 0.9206 - val_loss: 0.2123 - val_acc: 0.9209
Epoch 2/10
285/284 [==============================] - 354s - loss: 0.2101 - acc: 0.9220 - val_loss: 0.2362 - val_acc: 0.9096
Epoch 3/10
285/284 [==============================] - 354s - loss: 0.2078 - acc: 0.9235 - val_loss: 0.2136 - val_acc: 0.9187
Epoch 4/10
285/284 [==============================] - 354s - loss: 0.2050 - acc: 0.9239 - val_loss: 0.2211 - val_acc: 0.9182


In [10]:
y_pred = model.predict(x_val,batch_size=128)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val_w, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.926642198381
thresh: 0.1 	F2 score: 0.938457262846
thresh: 0.15 	F2 score: 0.939448346038
thresh: 0.2 	F2 score: 0.940934970826
thresh: 0.25 	F2 score: 0.939629211368
thresh: 0.3 	F2 score: 0.93520609825
thresh: 0.35 	F2 score: 0.932024044796


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [11]:
#continue with reduced learning rate
model.compile(loss='categorical_crossentropy', 
              optimizer=Adam(lr=0.0002),
              metrics=['accuracy']) 
model.fit_generator(datagen.flow(x_train,y_train_w, batch_size = 128), validation_data=(x_val, y_val_w),
                  epochs=10, steps_per_epoch=x_train.shape[0]/ 128, callbacks=callbacks,
                  )

Epoch 1/10
285/284 [==============================] - 356s - loss: 0.1931 - acc: 0.9281 - val_loss: 0.1947 - val_acc: 0.9244
Epoch 2/10
285/284 [==============================] - 354s - loss: 0.1898 - acc: 0.9279 - val_loss: 0.2015 - val_acc: 0.9266
Epoch 3/10
285/284 [==============================] - 354s - loss: 0.1899 - acc: 0.9282 - val_loss: 0.1975 - val_acc: 0.9289
Epoch 4/10
285/284 [==============================] - 354s - loss: 0.1851 - acc: 0.9311 - val_loss: 0.2428 - val_acc: 0.9153


In [12]:
y_pred = model.predict(x_val,batch_size=128)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val_w, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.924833545078
thresh: 0.1 	F2 score: 0.935592826558
thresh: 0.15 	F2 score: 0.938094061735
thresh: 0.2 	F2 score: 0.937211791831
thresh: 0.25 	F2 score: 0.934988471673
thresh: 0.3 	F2 score: 0.931488801054
thresh: 0.35 	F2 score: 0.92803030303


In [13]:
#continue with reduced learning rate
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(datagen.flow(x_train,y_train_w, batch_size = 128), validation_data=(x_val, y_val_w),
                  epochs=10, steps_per_epoch=x_train.shape[0]/ 128, callbacks=callbacks,
                  )

Epoch 1/10
285/284 [==============================] - 356s - loss: 0.1796 - acc: 0.9316 - val_loss: 0.1902 - val_acc: 0.9266
Epoch 2/10
285/284 [==============================] - 355s - loss: 0.1791 - acc: 0.9327 - val_loss: 0.1873 - val_acc: 0.9303
Epoch 3/10
285/284 [==============================] - 354s - loss: 0.1800 - acc: 0.9321 - val_loss: 0.1837 - val_acc: 0.9308
Epoch 4/10
285/284 [==============================] - 354s - loss: 0.1761 - acc: 0.9333 - val_loss: 0.1809 - val_acc: 0.9318
Epoch 5/10
285/284 [==============================] - 354s - loss: 0.1762 - acc: 0.9341 - val_loss: 0.1795 - val_acc: 0.9331
Epoch 6/10
285/284 [==============================] - 354s - loss: 0.1750 - acc: 0.9327 - val_loss: 0.1820 - val_acc: 0.9326
Epoch 7/10
285/284 [==============================] - 354s - loss: 0.1748 - acc: 0.9341 - val_loss: 0.1826 - val_acc: 0.9321
Epoch 8/10
285/284 [==============================] - 354s - loss: 0.1772 - acc: 0.9334 - val_loss: 0.1804 - val_acc: 0.9316


In [14]:
y_pred = model.predict(x_val,batch_size=128)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val_w, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.942992130152
thresh: 0.1 	F2 score: 0.951736895351
thresh: 0.15 	F2 score: 0.95275738754
thresh: 0.2 	F2 score: 0.951116365519
thresh: 0.25 	F2 score: 0.949251835121
thresh: 0.3 	F2 score: 0.947569640504
thresh: 0.35 	F2 score: 0.944499341238


In [15]:
#continue with reduced learning rate
model.compile(optimizer=Adam(lr=0.00005), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(datagen.flow(x_train,y_train_w, batch_size = 128), validation_data=(x_val, y_val_w),
                  epochs=10, steps_per_epoch=x_train.shape[0]/ 128, callbacks=callbacks,
                  )

Epoch 1/10
285/284 [==============================] - 356s - loss: 0.1724 - acc: 0.9347 - val_loss: 0.1773 - val_acc: 0.9326
Epoch 2/10
285/284 [==============================] - 354s - loss: 0.1710 - acc: 0.9345 - val_loss: 0.1812 - val_acc: 0.9316
Epoch 3/10
285/284 [==============================] - 354s - loss: 0.1693 - acc: 0.9360 - val_loss: 0.1813 - val_acc: 0.9321
Epoch 4/10
285/284 [==============================] - 354s - loss: 0.1708 - acc: 0.9346 - val_loss: 0.1806 - val_acc: 0.9328


In [16]:
y_pred = model.predict(x_val,batch_size=128)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val_w, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.946166831357
thresh: 0.1 	F2 score: 0.952141269057
thresh: 0.15 	F2 score: 0.952445652174
thresh: 0.2 	F2 score: 0.951039902127
thresh: 0.25 	F2 score: 0.950622294372
thresh: 0.3 	F2 score: 0.947628458498
thresh: 0.35 	F2 score: 0.946146245059


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [17]:
model.save("simple_96_weather")
model.save_weights("simple_96_weather_weights")

In [18]:
x_val = []
x_train = []
x_train = np.zeros((40479,96,96,3), np.float32)
y_train = []

df_train = pd.read_csv('train_v2.csv')

labels = ['blow_down',
 'bare_ground',
 'conventional_mine',
 'blooming',
 'artisinal_mine',
 'selective_logging',         
 'slash_burn', 
 'cultivation',
 'habitation',
 'road',
 'agriculture',
 'water',
 'primary',
 'partly_cloudy', 
 'cloudy',
 'clear',
 'haze',]

label_map = {l: i for i, l in enumerate(labels)}
inv_label_map = {i: l for l, i in label_map.items()}

i=0

for f, tags in tqdm(df_train.values[:40479], miniters=1000):    
    img = cv2.imread('train-jpg/{}.jpg'.format(f))
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
    x_train[i,:,:,:] = np.array(cv2.resize(img, (96, 96)),np.float32)/255.#139 minimum size for inception
    i+=1
    y_train.append(targets)
  
y_train = np.array(y_train, np.uint8)

print(x_train.shape)
print(y_train.shape)

100%|██████████| 40479/40479 [01:01<00:00, 663.00it/s]

(40479, 96, 96, 3)
(40479, 17)


In [19]:
y_train_2 = np.zeros((y_train.shape[0],7))
y_train_2[:,1:] = y_train[:,7:13]
y_train_2[:,0] = (np.sum(y_train[:,:7],axis=1)>0)
y_train_2 = np.array(y_train_2,np.uint8)

In [20]:
print(y_train[100,:])
print(y_train_2[100,:])

[0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0]
[1 0 0 0 0 1 0]


In [21]:
x_train -= train_mean
x_train, x_val, y_train_2, y_val_2 = train_test_split(x_train,y_train_2,test_size=0.1)
print(x_train.shape)
print(y_train_2.shape)
print(x_val.shape)
print(y_val_2.shape)

(36431, 96, 96, 3)
(36431, 7)
(4048, 96, 96, 3)
(4048, 7)


In [22]:
#model for the more common classes + 1 class for others
model1 = Sequential()
model1.add(Conv2D(32, (3, 3), padding = 'same', input_shape=(96,96,3)))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(Conv2D(48, (3, 3), padding = 'same'))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(Conv2D(48, (3, 3), padding = 'same'))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(Conv2D(48, (3, 3), padding = 'same'))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Conv2D(64, (3, 3), padding = 'same'))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(Conv2D(64, (3, 3), padding = 'same'))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(Conv2D(64, (3, 3), padding = 'same'))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Conv2D(128, (3, 3), padding = 'same'))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(Conv2D(128, (3, 3), padding = 'same'))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(Conv2D(128, (3, 3), padding = 'same'))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Flatten())
model1.add(Dense(2048))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(Dense(1024))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(Dense(7, activation='sigmoid'))

In [23]:
model1.compile(loss='binary_crossentropy', 
              optimizer='adam',
              metrics=['accuracy'])

model1.fit_generator(datagen.flow(x_train,y_train_2, batch_size = 128), validation_data=(x_val, y_val_2),
                   epochs=10, steps_per_epoch=x_train.shape[0]/ 128, callbacks=callbacks,
                  )

Epoch 1/10
285/284 [==============================] - 356s - loss: 0.2877 - acc: 0.8826 - val_loss: 0.5537 - val_acc: 0.8528
Epoch 2/10
285/284 [==============================] - 352s - loss: 0.2444 - acc: 0.9012 - val_loss: 0.2706 - val_acc: 0.8844
Epoch 3/10
285/284 [==============================] - 349s - loss: 0.2266 - acc: 0.9091 - val_loss: 0.2980 - val_acc: 0.8654
Epoch 4/10
285/284 [==============================] - 349s - loss: 0.2117 - acc: 0.9156 - val_loss: 0.2401 - val_acc: 0.9000
Epoch 5/10
285/284 [==============================] - 349s - loss: 0.2075 - acc: 0.9168 - val_loss: 0.2469 - val_acc: 0.9068
Epoch 6/10
285/284 [==============================] - 349s - loss: 0.2001 - acc: 0.9198 - val_loss: 0.2111 - val_acc: 0.9156
Epoch 7/10
285/284 [==============================] - 349s - loss: 0.1943 - acc: 0.9223 - val_loss: 0.2020 - val_acc: 0.9161
Epoch 8/10
285/284 [==============================] - 349s - loss: 0.1903 - acc: 0.9243 - val_loss: 0.1904 - val_acc: 0.9239


In [24]:
y_pred = model1.predict(x_val,batch_size=128)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val_2, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.817555602705
thresh: 0.1 	F2 score: 0.838127386563
thresh: 0.15 	F2 score: 0.845881131105
thresh: 0.2 	F2 score: 0.847182743305
thresh: 0.25 	F2 score: 0.846028920815
thresh: 0.3 	F2 score: 0.842465247655
thresh: 0.35 	F2 score: 0.840779675876


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


In [25]:
#continue with reduced learning rate
model1.compile(loss='binary_crossentropy', 
              optimizer=Adam(lr=0.0005),
              metrics=['accuracy']) 
model1.fit_generator(datagen.flow(x_train,y_train_2, batch_size = 128), validation_data=(x_val, y_val_2),
                   epochs=10, steps_per_epoch=x_train.shape[0]/ 128, callbacks=callbacks,
                  )

Epoch 1/10
285/284 [==============================] - 352s - loss: 0.1773 - acc: 0.9298 - val_loss: 0.1756 - val_acc: 0.9301
Epoch 2/10
285/284 [==============================] - 350s - loss: 0.1746 - acc: 0.9306 - val_loss: 0.2099 - val_acc: 0.9124
Epoch 3/10
285/284 [==============================] - 350s - loss: 0.1727 - acc: 0.9317 - val_loss: 0.2110 - val_acc: 0.9127
Epoch 4/10
285/284 [==============================] - 350s - loss: 0.1717 - acc: 0.9324 - val_loss: 0.1799 - val_acc: 0.9293


In [26]:
y_pred = model1.predict(x_val,batch_size=128)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val_2, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.839977856988
thresh: 0.1 	F2 score: 0.855837721125
thresh: 0.15 	F2 score: 0.86149630445
thresh: 0.2 	F2 score: 0.861781213679
thresh: 0.25 	F2 score: 0.858321488683
thresh: 0.3 	F2 score: 0.855323920893
thresh: 0.35 	F2 score: 0.851611453418


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


In [27]:
#continue with reduced learning rate
model1.compile(loss='binary_crossentropy', 
              optimizer=Adam(lr=0.0002),
              metrics=['accuracy']) 
model1.fit_generator(datagen.flow(x_train,y_train_2, batch_size = 128), validation_data=(x_val, y_val_2),
                   epochs=10, steps_per_epoch=x_train.shape[0]/ 128, callbacks=callbacks,
                  )

Epoch 1/10
285/284 [==============================] - 352s - loss: 0.1642 - acc: 0.9348 - val_loss: 0.1646 - val_acc: 0.9338
Epoch 2/10
285/284 [==============================] - 350s - loss: 0.1621 - acc: 0.9355 - val_loss: 0.1666 - val_acc: 0.9341
Epoch 3/10
285/284 [==============================] - 350s - loss: 0.1616 - acc: 0.9360 - val_loss: 0.1656 - val_acc: 0.9343
Epoch 4/10
285/284 [==============================] - 350s - loss: 0.1608 - acc: 0.9362 - val_loss: 0.1649 - val_acc: 0.9345


In [28]:
y_pred = model1.predict(x_val,batch_size=128)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val_2, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.843777893815
thresh: 0.1 	F2 score: 0.863830420275
thresh: 0.15 	F2 score: 0.871370555124
thresh: 0.2 	F2 score: 0.871896726532
thresh: 0.25 	F2 score: 0.869693272698
thresh: 0.3 	F2 score: 0.867344489821
thresh: 0.35 	F2 score: 0.863606839335


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


In [29]:
#continue with reduced learning rate
model1.compile(loss='binary_crossentropy', 
              optimizer=Adam(lr=0.0001),
              metrics=['accuracy']) 
model1.fit_generator(datagen.flow(x_train,y_train_2, batch_size = 128), validation_data=(x_val, y_val_2),
                  epochs=10, steps_per_epoch=x_train.shape[0]/ 128, callbacks=callbacks,
                  )

Epoch 1/10
285/284 [==============================] - 352s - loss: 0.1585 - acc: 0.9369 - val_loss: 0.1620 - val_acc: 0.9357
Epoch 2/10
285/284 [==============================] - 350s - loss: 0.1572 - acc: 0.9375 - val_loss: 0.1619 - val_acc: 0.9353
Epoch 3/10
285/284 [==============================] - 350s - loss: 0.1567 - acc: 0.9379 - val_loss: 0.1641 - val_acc: 0.9352
Epoch 4/10
285/284 [==============================] - 350s - loss: 0.1553 - acc: 0.9383 - val_loss: 0.1597 - val_acc: 0.9373
Epoch 5/10
285/284 [==============================] - 349s - loss: 0.1559 - acc: 0.9381 - val_loss: 0.1602 - val_acc: 0.9367
Epoch 6/10
285/284 [==============================] - 350s - loss: 0.1547 - acc: 0.9386 - val_loss: 0.1605 - val_acc: 0.9363
Epoch 7/10
285/284 [==============================] - 349s - loss: 0.1540 - acc: 0.9386 - val_loss: 0.1613 - val_acc: 0.9355


In [30]:
y_pred = model1.predict(x_val,batch_size=128)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val_2, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.848365967251
thresh: 0.1 	F2 score: 0.86647738212
thresh: 0.15 	F2 score: 0.87356236267
thresh: 0.2 	F2 score: 0.874282360978
thresh: 0.25 	F2 score: 0.874155810225
thresh: 0.3 	F2 score: 0.871668659311
thresh: 0.35 	F2 score: 0.867512456221


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


In [31]:
#continue with reduced learning rate
model1.compile(loss='binary_crossentropy', 
              optimizer=Adam(lr=0.00005),
              metrics=['accuracy']) 
model1.fit_generator(datagen.flow(x_train,y_train_2, batch_size = 128), validation_data=(x_val, y_val_2),
                   epochs=10, steps_per_epoch=x_train.shape[0]/ 128, callbacks=callbacks,
                  )

Epoch 1/10
285/284 [==============================] - 352s - loss: 0.1529 - acc: 0.9397 - val_loss: 0.1596 - val_acc: 0.9362
Epoch 2/10
285/284 [==============================] - 349s - loss: 0.1516 - acc: 0.9397 - val_loss: 0.1601 - val_acc: 0.9369
Epoch 3/10
285/284 [==============================] - 350s - loss: 0.1518 - acc: 0.9396 - val_loss: 0.1590 - val_acc: 0.9373
Epoch 4/10
285/284 [==============================] - 349s - loss: 0.1518 - acc: 0.9396 - val_loss: 0.1595 - val_acc: 0.9373
Epoch 5/10
285/284 [==============================] - 349s - loss: 0.1511 - acc: 0.9399 - val_loss: 0.1596 - val_acc: 0.9369
Epoch 6/10
285/284 [==============================] - 349s - loss: 0.1510 - acc: 0.9397 - val_loss: 0.1597 - val_acc: 0.9369


In [32]:
y_pred = model1.predict(x_val,batch_size=128)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val_2, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.851807361615
thresh: 0.1 	F2 score: 0.868370448257
thresh: 0.15 	F2 score: 0.874093301379
thresh: 0.2 	F2 score: 0.873533715989
thresh: 0.25 	F2 score: 0.871313971279
thresh: 0.3 	F2 score: 0.869116898615
thresh: 0.35 	F2 score: 0.86716280365


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


In [33]:
y_pred = model1.predict(x_val,batch_size=128)
bestthresh = 0
bestF2score = 0
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    F2score = fbeta_score(y_val_2, np.array(y_pred)>thresh, beta=2, average='samples')
    print("thresh:",thresh,"\tF2 score:",F2score)
    if F2score > bestF2score:
        bestthresh = thresh
        bestF2score = F2score        

thresh: 0.05 	F2 score: 0.851807361615
thresh: 0.1 	F2 score: 0.868370448257
thresh: 0.15 	F2 score: 0.874093301379
thresh: 0.2 	F2 score: 0.873533715989
thresh: 0.25 	F2 score: 0.871313971279
thresh: 0.3 	F2 score: 0.869116898615
thresh: 0.35 	F2 score: 0.86716280365


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


In [34]:
total,tp,tn,fp,fn = multilabelmetrics(y_val_2,np.array(y_pred)>bestthresh)
d = {'Total':total,'TP':tp,'TN':tn,'FP':fp,'FN':fn}
pd.DataFrame(d, index=['others']+labels[7:13])

,FN,FP,TN,TP,Total
others,82,180,3638,148,230
cultivation,102,457,3134,355,457
habitation,68,214,3464,302,370
road,44,403,2815,786,830
agriculture,59,477,2324,1188,1247
water,80,431,2891,646,726
primary,14,166,142,3726,3740


In [35]:
num_rare = np.sum(np.sum(y_train[:,:7],axis=1)>0)
print(num_rare)

2180


In [36]:
model1.save("simple_96_major")
model1.save_weights("simple_96_major_weights")

In [37]:
x_val = []
x_train = []
x_train = np.zeros((num_rare,96,96,3), np.float32)
y_train = []

df_train = pd.read_csv('train_v2.csv')

labels = ['blow_down',
 'bare_ground',
 'conventional_mine',
 'blooming',
 'artisinal_mine',
 'selective_logging',         
 'slash_burn', 
 'cultivation',
 'habitation',
 'road',
 'agriculture',
 'water',
 'primary',
 'partly_cloudy', 
 'cloudy',
 'clear',
 'haze',]

label_map = {l: i for i, l in enumerate(labels)}
inv_label_map = {i: l for l, i in label_map.items()}

i=0

for f, tags in tqdm(df_train.values[:40479], miniters=1000):    
    img = cv2.imread('train-jpg/{}.jpg'.format(f))
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
    if(np.sum(targets[:7])>0):
        x_train[i,:,:,:] = np.array(cv2.resize(img, (96, 96)),np.float32)/255.#139 minimum size for inception
        i+=1
        y_train.append(targets)
    
y_train = np.array(y_train, np.uint8)

print(x_train.shape)
print(y_train.shape)

100%|██████████| 40479/40479 [00:55<00:00, 723.51it/s]

(2180, 96, 96, 3)
(2180, 17)


In [38]:
x_train -= train_mean
x_train, x_val, y_train_3, y_val_3 = train_test_split(x_train,y_train[:,:7],test_size=0.1)
print(x_train.shape)
print(y_train_3.shape)
print(x_val.shape)
print(y_val_3.shape)

(1962, 96, 96, 3)
(1962, 7)
(218, 96, 96, 3)
(218, 7)


In [39]:
#model for the rarer classes
from keras.models import load_model
model2 = load_model("simple_96_major")
for layer in model2.layers[:-1]:
    layer.trainable = False
model2.layers[-1].trainable = True

In [40]:
model2.compile(loss='binary_crossentropy', 
              optimizer='adam',
              metrics=['accuracy'])

model2.fit_generator(datagen.flow(x_train,y_train_3, batch_size = 128), validation_data=(x_val, y_val_3),
                  epochs=10, steps_per_epoch=10*x_train.shape[0]/ 128, callbacks=callbacks,
                  )#more steps per epoch to compensate for fewer images

Epoch 1/10
154/153 [==============================] - 58s - loss: 0.5086 - acc: 0.8368 - val_loss: 0.2695 - val_acc: 0.8866
Epoch 2/10
154/153 [==============================] - 54s - loss: 0.2506 - acc: 0.9014 - val_loss: 0.2353 - val_acc: 0.8991
Epoch 3/10
154/153 [==============================] - 55s - loss: 0.2289 - acc: 0.9072 - val_loss: 0.2212 - val_acc: 0.9056
Epoch 4/10
154/153 [==============================] - 54s - loss: 0.2186 - acc: 0.9095 - val_loss: 0.2155 - val_acc: 0.9076
Epoch 5/10
154/153 [==============================] - 54s - loss: 0.2111 - acc: 0.9136 - val_loss: 0.2060 - val_acc: 0.9128
Epoch 6/10
154/153 [==============================] - 54s - loss: 0.2068 - acc: 0.9143 - val_loss: 0.2028 - val_acc: 0.9148
Epoch 7/10
154/153 [==============================] - 54s - loss: 0.2042 - acc: 0.9159 - val_loss: 0.2019 - val_acc: 0.9115
Epoch 8/10
154/153 [==============================] - 54s - loss: 0.2006 - acc: 0.9164 - val_loss: 0.2023 - val_acc: 0.9109
Epoch 9/

In [41]:
y_pred = model2.predict(x_val,batch_size=128)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val_3, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.753721367827
thresh: 0.1 	F2 score: 0.792183797344
thresh: 0.15 	F2 score: 0.779418529992
thresh: 0.2 	F2 score: 0.786814994384
thresh: 0.25 	F2 score: 0.766463213138
thresh: 0.3 	F2 score: 0.74739557079
thresh: 0.35 	F2 score: 0.715576727045


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [42]:
model2.compile(loss='binary_crossentropy', 
              optimizer=Adam(lr=0.0005),
              metrics=['accuracy']) 
model2.fit_generator(datagen.flow(x_train,y_train_3, batch_size = 128), validation_data=(x_val, y_val_3),
                  epochs=10, steps_per_epoch=10*x_train.shape[0]/ 128, callbacks=callbacks,
                  )

Epoch 1/10
154/153 [==============================] - 57s - loss: 0.1932 - acc: 0.9198 - val_loss: 0.1953 - val_acc: 0.9122
Epoch 2/10
154/153 [==============================] - 54s - loss: 0.1922 - acc: 0.9206 - val_loss: 0.1950 - val_acc: 0.9128
Epoch 3/10
154/153 [==============================] - 54s - loss: 0.1918 - acc: 0.9199 - val_loss: 0.1933 - val_acc: 0.9155
Epoch 4/10
154/153 [==============================] - 54s - loss: 0.1902 - acc: 0.9205 - val_loss: 0.1949 - val_acc: 0.9135
Epoch 5/10
154/153 [==============================] - 54s - loss: 0.1881 - acc: 0.9221 - val_loss: 0.1923 - val_acc: 0.9174
Epoch 6/10
154/153 [==============================] - 54s - loss: 0.1890 - acc: 0.9214 - val_loss: 0.1923 - val_acc: 0.9161
Epoch 7/10
154/153 [==============================] - 54s - loss: 0.1871 - acc: 0.9215 - val_loss: 0.1941 - val_acc: 0.9142
Epoch 8/10
154/153 [==============================] - 54s - loss: 0.1883 - acc: 0.9220 - val_loss: 0.1934 - val_acc: 0.9142


In [43]:
y_pred = model2.predict(x_val,batch_size=128)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val_3, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.757980042384
thresh: 0.1 	F2 score: 0.797852376981
thresh: 0.15 	F2 score: 0.791787468393
thresh: 0.2 	F2 score: 0.779361438765
thresh: 0.25 	F2 score: 0.765889818642
thresh: 0.3 	F2 score: 0.748196502784
thresh: 0.35 	F2 score: 0.721692934996


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [44]:
model2.compile(loss='binary_crossentropy', 
              optimizer=Adam(lr=0.0002),
              metrics=['accuracy']) 
model2.fit_generator(datagen.flow(x_train,y_train_3, batch_size = 128), validation_data=(x_val, y_val_3),
                  epochs=10, steps_per_epoch=10*x_train.shape[0]/ 128, callbacks=callbacks,
                  )

Epoch 1/10
154/153 [==============================] - 57s - loss: 0.1855 - acc: 0.9228 - val_loss: 0.1929 - val_acc: 0.9155
Epoch 2/10
154/153 [==============================] - 54s - loss: 0.1865 - acc: 0.9219 - val_loss: 0.1931 - val_acc: 0.9148
Epoch 3/10
154/153 [==============================] - 54s - loss: 0.1865 - acc: 0.9221 - val_loss: 0.1931 - val_acc: 0.9161
Epoch 4/10
154/153 [==============================] - 54s - loss: 0.1864 - acc: 0.9224 - val_loss: 0.1928 - val_acc: 0.9142
Epoch 5/10
154/153 [==============================] - 54s - loss: 0.1865 - acc: 0.9225 - val_loss: 0.1930 - val_acc: 0.9142
Epoch 6/10
154/153 [==============================] - 54s - loss: 0.1850 - acc: 0.9229 - val_loss: 0.1921 - val_acc: 0.9148
Epoch 7/10
154/153 [==============================] - 54s - loss: 0.1851 - acc: 0.9227 - val_loss: 0.1916 - val_acc: 0.9161
Epoch 8/10
154/153 [==============================] - 54s - loss: 0.1850 - acc: 0.9231 - val_loss: 0.1924 - val_acc: 0.9155
Epoch 9/

In [45]:
y_pred = model2.predict(x_val,batch_size=128)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val_3, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.759332347062
thresh: 0.1 	F2 score: 0.792136635027
thresh: 0.15 	F2 score: 0.790059838225
thresh: 0.2 	F2 score: 0.782419542741
thresh: 0.25 	F2 score: 0.765889818642
thresh: 0.3 	F2 score: 0.751727884755
thresh: 0.35 	F2 score: 0.726025248961


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [46]:
model2.compile(loss='binary_crossentropy', 
              optimizer=Adam(lr=0.0001),
              metrics=['accuracy']) 
model2.fit_generator(datagen.flow(x_train,y_train_3, batch_size = 128), validation_data=(x_val, y_val_3),
                  epochs=10, steps_per_epoch=10*x_train.shape[0]/ 128, callbacks=callbacks,
                  )

Epoch 1/10
154/153 [==============================] - 57s - loss: 0.1831 - acc: 0.9234 - val_loss: 0.1917 - val_acc: 0.9155
Epoch 2/10
154/153 [==============================] - 54s - loss: 0.1841 - acc: 0.9231 - val_loss: 0.1918 - val_acc: 0.9161
Epoch 3/10
154/153 [==============================] - 54s - loss: 0.1843 - acc: 0.9235 - val_loss: 0.1920 - val_acc: 0.9155
Epoch 4/10
154/153 [==============================] - 54s - loss: 0.1832 - acc: 0.9236 - val_loss: 0.1917 - val_acc: 0.9168
Epoch 5/10
154/153 [==============================] - 54s - loss: 0.1834 - acc: 0.9230 - val_loss: 0.1915 - val_acc: 0.9148
Epoch 6/10
154/153 [==============================] - 55s - loss: 0.1835 - acc: 0.9238 - val_loss: 0.1915 - val_acc: 0.9142
Epoch 7/10
154/153 [==============================] - 54s - loss: 0.1833 - acc: 0.9236 - val_loss: 0.1921 - val_acc: 0.9161
Epoch 8/10
154/153 [==============================] - 54s - loss: 0.1831 - acc: 0.9235 - val_loss: 0.1920 - val_acc: 0.9161
Epoch 9/

In [47]:
y_pred = model2.predict(x_val,batch_size=128)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val_3, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.759632696559
thresh: 0.1 	F2 score: 0.797815970981
thresh: 0.15 	F2 score: 0.787902782742
thresh: 0.2 	F2 score: 0.782829110237
thresh: 0.25 	F2 score: 0.772006026593
thresh: 0.3 	F2 score: 0.750963358762
thresh: 0.35 	F2 score: 0.720673567004


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [48]:
model2.compile(loss='binary_crossentropy', 
              optimizer=Adam(lr=0.00005),
              metrics=['accuracy']) 
model2.fit_generator(datagen.flow(x_train,y_train_3, batch_size = 128), validation_data=(x_val, y_val_3),
                  epochs=10, steps_per_epoch=10*x_train.shape[0]/ 128, callbacks=callbacks,
                  )

Epoch 1/10
154/153 [==============================] - 57s - loss: 0.1827 - acc: 0.9237 - val_loss: 0.1915 - val_acc: 0.9148
Epoch 2/10
154/153 [==============================] - 54s - loss: 0.1822 - acc: 0.9240 - val_loss: 0.1913 - val_acc: 0.9155
Epoch 3/10
154/153 [==============================] - 54s - loss: 0.1841 - acc: 0.9230 - val_loss: 0.1912 - val_acc: 0.9155
Epoch 4/10
154/153 [==============================] - 54s - loss: 0.1822 - acc: 0.9246 - val_loss: 0.1912 - val_acc: 0.9161
Epoch 5/10
154/153 [==============================] - 54s - loss: 0.1826 - acc: 0.9241 - val_loss: 0.1915 - val_acc: 0.9161
Epoch 6/10
154/153 [==============================] - 55s - loss: 0.1824 - acc: 0.9242 - val_loss: 0.1916 - val_acc: 0.9155


In [49]:
y_pred = model2.predict(x_val,batch_size=128)
bestthresh = 0
bestF2score = 0
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    F2score = fbeta_score(y_val_3, np.array(y_pred)>thresh, beta=2, average='samples')
    print("thresh:",thresh,"\tF2 score:",F2score)
    if F2score > bestF2score:
        bestthresh = thresh
        bestF2score = F2score    

thresh: 0.05 	F2 score: 0.757028012762
thresh: 0.1 	F2 score: 0.806580715411
thresh: 0.15 	F2 score: 0.786046076757
thresh: 0.2 	F2 score: 0.783375200233
thresh: 0.25 	F2 score: 0.770476974605
thresh: 0.3 	F2 score: 0.747905254786
thresh: 0.35 	F2 score: 0.725260722967


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [50]:
total,tp,tn,fp,fn = multilabelmetrics(y_val_3,np.array(y_pred)>bestthresh)
d = {'Total':total,'TP':tp,'TN':tn,'FP':fp,'FN':fn}
pd.DataFrame(d, index=labels[:7])

,FN,FP,TN,TP,Total
blow_down,3,14,194,7,10
bare_ground,2,70,67,79,81
conventional_mine,2,11,196,9,11
blooming,1,32,154,31,32
artisinal_mine,4,25,155,34,38
selective_logging,1,48,131,38,39
slash_burn,0,33,165,20,20


In [51]:
model2.save("simple_96_rare")
model2.save_weights("simple_96_rare_weights")

In [52]:
#F2 score on training set
x_val = []
x_train = []
x_train = np.zeros((40479,96,96,3), np.float32)
y_train = []

df_train = pd.read_csv('train_v2.csv')

labels = ['blow_down',
 'bare_ground',
 'conventional_mine',
 'blooming',
 'artisinal_mine',
 'selective_logging',         
 'slash_burn', 
 'cultivation',
 'habitation',
 'road',
 'agriculture',
 'water',
 'primary',
 'partly_cloudy', 
 'cloudy',
 'clear',
 'haze',]

label_map = {l: i for i, l in enumerate(labels)}
inv_label_map = {i: l for l, i in label_map.items()}

i=0

for f, tags in tqdm(df_train.values[:40479], miniters=1000):    
    img = cv2.imread('train-jpg/{}.jpg'.format(f))
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
    x_train[i,:,:,:] = np.array(cv2.resize(img, (96, 96)),np.float32)/255.#139 minimum size for inception
    i+=1
    y_train.append(targets)

y_train = np.array(y_train, np.uint8)

print(x_train.shape)
print(y_train.shape)

100%|██████████| 40479/40479 [01:01<00:00, 657.84it/s]

(40479, 96, 96, 3)
(40479, 17)


In [53]:
x_train -= train_mean

In [54]:
y = model.predict(x_train,batch_size=128)
y1 = model1.predict(x_train,batch_size=128)
y2 = model2.predict(x_train,batch_size=128)

In [55]:
print(y.shape)
print(y1.shape)
print(y2.shape)

(40479, 4)
(40479, 7)
(40479, 7)


In [58]:
f2scorelist = []
for thresh in [0.1,0.15,0.2,0.25]:
    for thresh1 in [0.1,0.15,0.2,0.25,0.3]:
        for thresh2 in [0.15,0.2,0.25,0.3,0.35]:
            for thresh3 in [0.15,0.2,0.25,0.3,0.35]:
                for thresh4 in [0.55,0.6,0.65,0.7,0.75]:
                    y_pred = combine_predictions_2(x_train,y,y1,y2,thresh,thresh1,thresh2,thresh3,thresh4)
                    f2scorelist.append([thresh,thresh1,thresh2,thresh3,thresh4,fbeta_score(y_train,y_pred , beta=2, average='samples')])

In [59]:
f2scorelist.sort(key=lambda x: x[5],reverse=True)
pd.DataFrame(f2scorelist[:10])

,0,1,2,3,4,5
0,0.15,0.2,0.35,0.25,0.75,0.930303
1,0.15,0.2,0.35,0.25,0.70,0.930296
2,0.15,0.2,0.35,0.20,0.75,0.930296
3,0.15,0.2,0.35,0.20,0.70,0.930289
4,0.15,0.2,0.30,0.25,0.75,0.930272
5,0.15,0.2,0.25,0.25,0.75,0.930266
6,0.15,0.2,0.30,0.25,0.70,0.930265
7,0.15,0.2,0.25,0.25,0.70,0.930259
8,0.15,0.2,0.30,0.20,0.75,0.930227
9,0.15,0.2,0.30,0.20,0.70,0.930220


In [60]:
f2scorelistfiner = []
for thresh in [0.12,0.14,0.15,0.16,0.18,0.2,0.22]:
    for thresh1 in [0.16,0.18,0.2,0.22,0.24]:
        for thresh2 in [0.225,0.25,0.275,0.3,0.325,0.35,0.375,0.4]:
            for thresh3 in [0.15,0.175,0.2,0.225,0.25,0.275]:
                for thresh4 in [0.675,0.7,0.725,0.75,0.775,0.8]:
                    y_pred = combine_predictions_2(x_train,y,y1,y2,thresh,thresh1,thresh2,thresh3,thresh4)
                    f2scorelistfiner.append([thresh,thresh1,thresh2,thresh3,thresh4,fbeta_score(y_train,y_pred , beta=2, average='samples')])

In [62]:
f2scorelistfiner.sort(key=lambda x: x[5],reverse=True)
pd.DataFrame(f2scorelistfiner[:10])

,0,1,2,3,4,5
0,0.15,0.22,0.400,0.200,0.800,0.930454
1,0.15,0.22,0.375,0.200,0.800,0.930443
2,0.15,0.22,0.400,0.200,0.675,0.930434
3,0.15,0.22,0.400,0.225,0.800,0.930434
4,0.15,0.22,0.375,0.250,0.800,0.930428
5,0.15,0.22,0.400,0.250,0.800,0.930426
6,0.15,0.22,0.375,0.225,0.800,0.930425
7,0.15,0.22,0.375,0.200,0.675,0.930423
8,0.14,0.22,0.400,0.200,0.800,0.930421
9,0.15,0.22,0.350,0.250,0.800,0.930418


In [64]:
f2scorelistfiner = []
for thresh in [0.13,0.14,0.15,0.16]:
    for thresh1 in [0.22]:
        for thresh2 in [0.375,0.4]:
            for thresh3 in [0.2,0.225,0.25]:
                for thresh4 in [0.675,0.7,0.725,0.75,0.775,0.8,0.825,0.85]:
                    y_pred = combine_predictions_2(x_train,y,y1,y2,thresh,thresh1,thresh2,thresh3,thresh4)
                    f2scorelistfiner.append([thresh,thresh1,thresh2,thresh3,thresh4,fbeta_score(y_train,y_pred , beta=2, average='samples')])

In [65]:
f2scorelistfiner.sort(key=lambda x: x[5],reverse=True)
pd.DataFrame(f2scorelistfiner[:10])

,0,1,2,3,4,5
0,0.15,0.22,0.400,0.200,0.800,0.930454
1,0.15,0.22,0.375,0.200,0.800,0.930443
2,0.15,0.22,0.400,0.200,0.675,0.930434
3,0.15,0.22,0.400,0.225,0.800,0.930434
4,0.15,0.22,0.400,0.200,0.825,0.930431
5,0.15,0.22,0.375,0.250,0.800,0.930428
6,0.15,0.22,0.400,0.250,0.800,0.930426
7,0.15,0.22,0.375,0.225,0.800,0.930425
8,0.15,0.22,0.375,0.200,0.675,0.930423
9,0.14,0.22,0.400,0.200,0.800,0.930421


In [66]:
#Test set
x_train = []
x_val = []
x_test = np.zeros((61191,96,96,3), np.float32)
y_train = []

df_test = pd.read_csv('sample_submission_v2.csv')

i = 0 
for f, tags in tqdm(df_test.values, miniters=1000):
    img = cv2.imread('test-jpg/{}.jpg'.format(f))
    x_test[i,:,:,:] = np.array(cv2.resize(img, (96, 96)),np.float32)/255.#139 minimum size for inception
    i+=1
print(x_test.shape)

100%|██████████| 61191/61191 [09:18<00:00, 109.55it/s]

(61191, 96, 96, 3)


In [67]:
x_test -= train_mean

y = model.predict(x_test,batch_size=128)
y1 = model1.predict(x_test,batch_size=128)
y2 = model2.predict(x_test,batch_size=128)

In [68]:
thresh,thresh1,thresh2,thresh3,thresh4 = f2scorelistfiner[0][:5]
y_pred = combine_predictions_2(x_test,y,y1,y2,thresh,thresh1,thresh2,thresh3,thresh4)
print(y_pred.shape)

(61191, 17)


In [69]:
labels_np = np.array(labels)
preds = [' '.join(labels_np[np.array(y_pred[i,:],bool)]) for i in range(y_pred.shape[0])]
subm = pd.DataFrame()
subm['image_name'] = df_test.values[:,0]
subm['tags'] = preds
subm.to_csv('submission_96_3net_1.csv', index=False)
#test set score:0.92451

In [70]:
thresh,thresh1,thresh2,thresh3,thresh4 = f2scorelistfiner[1][:5]
y_pred = combine_predictions_2(x_test,y,y1,y2,thresh,thresh1,thresh2,thresh3,thresh4)
print(y_pred.shape)

(61191, 17)


In [71]:
labels_np = np.array(labels)
preds = [' '.join(labels_np[np.array(y_pred[i,:],bool)]) for i in range(y_pred.shape[0])]
subm = pd.DataFrame()
subm['image_name'] = df_test.values[:,0]
subm['tags'] = preds
subm.to_csv('submission_96_3net_2.csv', index=False)
#test set score:0.92445

In [72]:
thresh,thresh1,thresh2,thresh3,thresh4 = f2scorelistfiner[2][:5]
y_pred = combine_predictions_2(x_test,y,y1,y2,thresh,thresh1,thresh2,thresh3,thresh4)
print(y_pred.shape)

(61191, 17)


In [73]:
labels_np = np.array(labels)
preds = [' '.join(labels_np[np.array(y_pred[i,:],bool)]) for i in range(y_pred.shape[0])]
subm = pd.DataFrame()
subm['image_name'] = df_test.values[:,0]
subm['tags'] = preds
subm.to_csv('submission_96_3net_3.csv', index=False)
#test set score:0.92446

In [74]:
thresh,thresh1,thresh2,thresh3,thresh4 = f2scorelistfiner[3][:5]
y_pred = combine_predictions_2(x_test,y,y1,y2,thresh,thresh1,thresh2,thresh3,thresh4)
print(y_pred.shape)

(61191, 17)


In [75]:
labels_np = np.array(labels)
preds = [' '.join(labels_np[np.array(y_pred[i,:],bool)]) for i in range(y_pred.shape[0])]
subm = pd.DataFrame()
subm['image_name'] = df_test.values[:,0]
subm['tags'] = preds
subm.to_csv('submission_96_3net_4.csv', index=False)
#test set score: 0.92457

In [76]:
thresh,thresh1,thresh2,thresh3,thresh4 = f2scorelistfiner[4][:5]
y_pred = combine_predictions_2(x_test,y,y1,y2,thresh,thresh1,thresh2,thresh3,thresh4)
print(y_pred.shape)

(61191, 17)


In [77]:
labels_np = np.array(labels)
preds = [' '.join(labels_np[np.array(y_pred[i,:],bool)]) for i in range(y_pred.shape[0])]
subm = pd.DataFrame()
subm['image_name'] = df_test.values[:,0]
subm['tags'] = preds
subm.to_csv('submission_96_3net_5.csv', index=False)
#test set score: 0.92455